In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
ruta_csv = "/content/drive/MyDrive/3 - Master Computacion y Sistemas Inteligentes/PFM/Datasets/dataset_empresa_03_capacidad_buques.csv"

dtype_especifico = {
    "con_clave": "Int64",
    "con_bacoope": "Int64",
    "con_fecha": "str",
    "con_tipo": "string",
    "con_codigo": "string",
    "con_naviera": "string",
    "con_dni": "string",
    "con_tipobillete": "string",
    "con_cupon": "string",
    "con_trayecto": "string",
    "con_shortcomp": "string",
    "con_dateticket": "str",
    "con_acredita": "string",
    "con_nombre": "string",
    "con_intercambiable": "Int64",
    "con_incidencia": "string",
    "buq_nombre": "string",
    "capacidad_pasajeros": "Int64",
    "capacidad_vehiculos": "Int64"
}

df = pd.read_csv(ruta_csv, dtype=dtype_especifico, low_memory=False)
df["con_fecha"] = pd.to_datetime(df["con_fecha"], errors="coerce")
df["con_dateticket"] = pd.to_datetime(df["con_dateticket"], errors="coerce")
df.info()

In [ ]:
df.head()

In [ ]:
num_valores_unicos = df["con_incidencia"].nunique(dropna=False)

valores_unicos = df["con_incidencia"].unique()

num_valores_unicos, valores_unicos
df["con_incidencia"].value_counts(dropna=False)

In [ ]:

num_total_registros = df.shape[0]

df_filtrado = df[df["con_incidencia"] != "DESEMBARCADO"]

num_registros_sin_desembarcado = df_filtrado.shape[0]

print(f"Total de registros en el dataset: {num_total_registros}")
print(f"Total de registros sin 'DESEMBARCADO' en con_incidencia: {num_registros_sin_desembarcado}")

In [ ]:

df["con_desembarcado"] = df["con_incidencia"].fillna("").eq("DESEMBARCADO").astype(int)

df.drop(columns=["con_incidencia"], inplace=True)

df.head()

In [ ]:

num_valores_unicos = df["con_desembarcado"].nunique(dropna=False)

valores_unicos = df["con_desembarcado"].unique()

num_valores_unicos, valores_unicos

df["con_desembarcado"].value_counts(dropna=False)

In [ ]:

df = df.query("con_desembarcado != 1")

In [ ]:

df["anio_embarque"] = df["con_fecha"].dt.year

df_desglose = df.groupby("anio_embarque").agg(
    total_pasajeros = ("con_tipobillete", lambda x: x.isin(['1', '2', '3']).sum()),
    total_vehiculos = ("con_tipobillete", lambda x: x.isin(['0', '5', '6', '7', '8', '9']).sum()),
    total_mascotas = ("con_tipobillete", lambda x: x.isin(['M']).sum()),
    total_chofers = ("con_tipobillete", lambda x: x.isin(['4']).sum())
).reset_index()

display(df_desglose)

In [ ]:

df = df[~df["con_tipobillete"].isin(["4", "M"])]
print("Distribucion de con_tipobillete despues de eliminar '4' y 'M':")
print(df["con_tipobillete"].value_counts())

In [ ]:

df["anio"] = df["con_fecha"].dt.year

df_desglose = df.groupby("anio").agg(
    total_pasajeros = ("con_tipobillete", lambda x: x.isin(['1', '2', '3']).sum()),
    total_vehiculos = ("con_tipobillete", lambda x: x.isin(['0', '5', '6', '7', '8', '9']).sum()),
    total_mascotas = ("con_tipobillete", lambda x: x.isin(['M']).sum()),
    total_chofers = ("con_tipobillete", lambda x: x.isin(['4']).sum())
).reset_index()
display(df_desglose)

In [ ]:

num_total_registros = df.shape[0]
print(f"Total de registros en el dataset: {num_total_registros}")

In [ ]:

df_billetes = df.groupby([df["con_fecha"].dt.year, "con_tipobillete"]).size().unstack(fill_value=0).reset_index()

df_billetes.columns.name = None  
df_billetes.rename(columns={
    "0": "total_vehiculos_0",
    "1": "total_pasajeros_1",
    "2": "total_pasajeros_2",
    "3": "total_pasajeros_3",
    "4": "total_chofers",
    "5": "total_vehiculos_5",
    "6": "total_vehiculos_6",
    "7": "total_vehiculos_7",
    "8": "total_vehiculos_8",
    "9": "total_vehiculos_9",
    "M": "total_mascotas"
}, inplace=True)

display(df_billetes)

In [ ]:

df_billetes = df.pivot_table(index=df["con_fecha"].dt.year,
                             columns="con_tipobillete",
                             aggfunc="size",
                             fill_value=0).reset_index()


df_billetes.columns.name = None  

nombres_columnas = {
    "0": "total_vehiculos_0",
    "1": "total_pasajeros_1",
    "2": "total_pasajeros_2",
    "3": "total_pasajeros_3",
    "4": "total_chofers",
    "5": "total_vehiculos_5",
    "6": "total_vehiculos_6",
    "7": "total_vehiculos_7",
    "8": "total_vehiculos_8",
    "9": "total_vehiculos_9",
    "M": "total_mascotas"
}


df_billetes.rename(columns={k: v for k, v in nombres_columnas.items() if k in df_billetes.columns}, inplace=True)


columnas_billetes = [col for col in df_billetes.columns if col.startswith("total_vehiculos_") or
                      col.startswith("total_pasajeros_") or col.startswith("total_chofers") or
                      col.startswith("total_mascotas")]


df_billetes["total_pasajeros"] = df_billetes[[col for col in columnas_billetes if col.startswith("total_pasajeros_")]].sum(axis=1)
df_billetes["total_vehiculos"] = df_billetes[[col for col in columnas_billetes if col.startswith("total_vehiculos_")]].sum(axis=1)

df_billetes["total_anual"] = df_billetes[columnas_billetes].sum(axis=1)

display(df_billetes)


In [ ]:

columnas_df = df.columns.tolist()

num_total_registros = df.shape[0]

print(f"Total de registros en el dataset: {num_total_registros}")
print(f"Columnas del dataset: {columnas_df}")
display(df.head())
df.info()


In [ ]:

conteo_intercambiable = df["con_intercambiable"].value_counts()

conteo_desembarcado = df["con_desembarcado"].value_counts()
print("Conteo de registros en 'con_intercambiable':")
print(conteo_intercambiable)
print("Conteo de registros en 'con_desembarcado':")
print(conteo_desembarcado)

## **A partir de aqui elimino los registros de desembarcados**

In [ ]:

if 'con_desembarcado' in df.columns:
 
    df = df[df['con_desembarcado'] != 1]
   
    df = df.drop(columns=['con_desembarcado'])
print(f"Total de registros después de la limpieza: {df.shape[0]}")

In [ ]:

duplicados = df[df.duplicated(subset=['con_codigo', 'con_cupon', 'con_tipobillete'], keep=False)]

num_duplicados = duplicados.shape[0]
from IPython.display import display

print(f"Se encontraron {num_duplicados} registros duplicados en la combinacion de con_codigo, con_cupon y con_tipobillete.")

if not duplicados.empty:
    display(duplicados)
else:
    print("No se encontraron registros duplicados.")

In [ ]:

df = df.drop_duplicates(subset=['con_codigo', 'con_cupon', 'con_tipobillete'], keep='last')

In [ ]:

duplicados = df[df.duplicated(subset=['con_codigo', 'con_cupon', 'con_tipobillete'], keep=False)]

num_duplicados = duplicados.shape[0]
from IPython.display import display

print(f"Se encontraron {num_duplicados} registros duplicados en la combinacion de con_codigo, con_cupon y con_tipobillete.")

if not duplicados.empty:
    display(duplicados)
else:
    print("No se encontraron registros duplicados.")

In [ ]:

valores_validos = {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'M'}

valores_invalidos = df[~df['con_tipobillete'].astype(str).isin(valores_validos)]

num_invalidos = valores_invalidos.shape[0]
from IPython.display import display
print(f"Se encontraron {num_invalidos} registros con valores invalidos en con_tipobillete.")

if not valores_invalidos.empty:
    display(valores_invalidos)
else:
    print("Todos los valores de con_tipobillete son validos.")

In [ ]:

columnas_a_verificar = ['con_clave', 'con_bacoope', 'con_fecha', 'con_tipo', 'con_codigo',
                        'con_naviera', 'con_tipobillete', 'con_cupon', 'con_trayecto',
                        'con_shortcomp', 'con_dateticket']

valores_vacios = df[columnas_a_verificar].isnull().sum()

valores_vacios = valores_vacios[valores_vacios > 0]
from IPython.display import display

if not valores_vacios.empty:
    print("Se encontraron valores vacios en las siguientes columnas:")
    display(valores_vacios)
else:
    print("No hay valores vacios en las columnas seleccionadas.")

In [ ]:

tipos_fechas = df[["con_fecha", "con_dateticket"]].dtypes
print("Tipos de datos de las columnas con fechas:")
print(tipos_fechas)


In [ ]:

df["con_fecha"] = pd.to_datetime(df["con_fecha"], errors="coerce")
df["con_dateticket"] = pd.to_datetime(df["con_dateticket"], errors="coerce")

df["diferencia_min"] = (df["con_fecha"] - df["con_dateticket"]).dt.total_seconds() / 60

df.drop(columns=["diferencia_tiempo_hhmm"], errors="ignore", inplace=True)

df_resultado = df[["con_fecha", "con_dateticket", "diferencia_min"]]
display(df_resultado.head(10))

In [ ]:

df.head(100)

In [ ]:

num_total_registros = df.shape[0]
print(f"Total de registros en el dataset: {num_total_registros}")

In [ ]:

total_antes = (df["diferencia_min"] < 0).sum()  
total_despues = (df["diferencia_min"] > 0).sum() 
total_exacto = (df["diferencia_min"] == 0).sum() 
print(f"Total de lecturas ANTES de la hora de zarpado: {total_antes}")
print(f"Total de lecturas DESPUES de la hora de zarpado: {total_despues}")
print(f"Total de lecturas EXACTAMENTE a la hora de zarpado: {total_exacto}")

In [ ]:
if hasattr(df["con_fecha"].dt, "tz"):
    print("\nZona horaria detectada en 'con_fecha':", df["con_fecha"].dt.tz)
    print("Zona horaria detectada en 'con_dateticket':", df["con_dateticket"].dt.tz)
else:
    print("\nLas columnas de fecha no tienen zona horaria asignada.")

fecha_min = df[["con_fecha", "con_dateticket"]].min().min()
fecha_max = df[["con_fecha", "con_dateticket"]].max().max()

fecha_limite_min = pd.Timestamp("2000-01-01")
fecha_limite_max = pd.Timestamp("2050-12-31")

fechas_fuera_rango = df[
    (df["con_fecha"] < fecha_limite_min) | (df["con_fecha"] > fecha_limite_max) |
    (df["con_dateticket"] < fecha_limite_min) | (df["con_dateticket"] > fecha_limite_max)
]
print(f"\nFecha minima detectada: {fecha_min}")
print(f"Fecha maxima detectada: {fecha_max}")
print(f"\nTotal de registros con fechas fuera del rango esperado: {len(fechas_fuera_rango)}")

if not fechas_fuera_rango.empty:
    from IPython.display import display
    display(fechas_fuera_rango.head(10))

In [ ]:

columnas_categoricas = ["con_naviera", "con_trayecto", "con_tipo", "con_shortcomp"]
for col in columnas_categoricas:
    valores_unicos = df[col].dropna().unique()
    print(f"\nValores únicos en '{col}':")
    print(valores_unicos)

# Creo una copia del dataset original para seguir trabajando desde este punto

In [ ]:
import pandas as pd

df_copy = df.copy()

In [ ]:

df_copy.loc[df_copy['buq_nombre'] == "AL ANDALUS EXPRESS", 'capacidad_pasajeros'] = 215

df_copy.loc[df_copy['buq_nombre'] == "AL ANDALUS EXPRESS", 'capacidad_vehiculos'] = 150

df_copy.loc[df_copy['buq_nombre'] == "CECILIA PAYNE", 'capacidad_pasajeros'] = 800
df_copy.loc[df_copy['buq_nombre'] == "CECILIA PAYNE", 'capacidad_vehiculos'] = 200

df_copy.loc[df_copy['buq_nombre'] == "HELLENIC HIGHSPEED", 'capacidad_pasajeros'] = 727
df_copy.loc[df_copy['buq_nombre'] == "HELLENIC HIGHSPEED", 'capacidad_vehiculos'] = 175

df_copy.loc[df_copy['buq_nombre'] == "JAUME I", 'capacidad_pasajeros'] = 655
df_copy.loc[df_copy['buq_nombre'] == "JAUME I", 'capacidad_vehiculos'] = 144

df_copy.loc[df_copy['buq_nombre'] == "KATTEGAT", 'capacidad_pasajeros'] = 974
df_copy.loc[df_copy['buq_nombre'] == "KATTEGAT", 'capacidad_vehiculos'] = 344

df_copy.loc[df_copy['buq_nombre'] == "LEVANTE JET", 'capacidad_pasajeros'] = 675
df_copy.loc[df_copy['buq_nombre'] == "LEVANTE JET", 'capacidad_vehiculos'] = 151

df_copy.loc[df_copy['buq_nombre'] == "NISSOS CHIOS", 'capacidad_pasajeros'] = 1750
df_copy.loc[df_copy['buq_nombre'] == "NISSOS CHIOS", 'capacidad_vehiculos'] = 425

df_copy.loc[df_copy['buq_nombre'] == "PATRIA SEAWAYS", 'capacidad_pasajeros'] = 260
df_copy.loc[df_copy['buq_nombre'] == "PATRIA SEAWAYS", 'capacidad_vehiculos'] = 480

df_copy.loc[df_copy['buq_nombre'] == "STENA EUROPE", 'capacidad_pasajeros'] = 1386
df_copy.loc[df_copy['buq_nombre'] == "STENA EUROPE", 'capacidad_vehiculos'] = 564

df_copy.loc[df_copy['buq_nombre'] == "TARIFA JET", 'capacidad_pasajeros'] = 800
df_copy.loc[df_copy['buq_nombre'] == "TARIFA JET", 'capacidad_vehiculos'] = 175

df_copy.loc[df_copy['buq_nombre'] == "VOLCAN DE TAMASITE", 'capacidad_pasajeros'] = 1500
df_copy.loc[df_copy['buq_nombre'] == "VOLCAN DE TAMASITE", 'capacidad_vehiculos'] = 300

df_copy.loc[df_copy['buq_nombre'] == "VOLCAN DE TAUCE", 'capacidad_pasajeros'] = 347
df_copy.loc[df_copy['buq_nombre'] == "VOLCAN DE TAUCE", 'capacidad_vehiculos'] = 320

df_copy.loc[df_copy['buq_nombre'] == "WASA EXPRESS", 'capacidad_pasajeros'] = 1500
df_copy.loc[df_copy['buq_nombre'] == "WASA EXPRESS", 'capacidad_vehiculos'] = 450

In [ ]:

tipos_pasajeros = ['1', '2', '3']
tipos_vehiculos = ['0', '5', '6', '7', '8', '9']

totales = df_copy.groupby('con_bacoope').apply(lambda grupo: pd.Series({
    'total_pasajeros': grupo['con_tipobillete'].isin(tipos_pasajeros).sum(),
    'total_vehiculos': grupo['con_tipobillete'].isin(tipos_vehiculos).sum()
})).reset_index()

df_copy = pd.merge(df_copy, totales, on='con_bacoope', how='left')

df_copy['porc_ocupacion_pasajeros'] = (df_copy['total_pasajeros'] / df_copy['capacidad_pasajeros']) * 100
df_copy['porc_ocupacion_vehiculos'] = (df_copy['total_vehiculos'] / df_copy['capacidad_vehiculos']) * 100

df_copy.head(100)


In [ ]:

buques_capacidades = df_copy.groupby("buq_nombre")[["capacidad_pasajeros", "capacidad_vehiculos"]].max()
print("Capacidad de cada buque (pasajeros y vehiculos):")
print(buques_capacidades)

In [ ]:
import pandas as pd

resumen_operativas = df_copy.groupby(['con_bacoope', 'buq_nombre']).agg({
    'porc_ocupacion_pasajeros': 'max',
    'porc_ocupacion_vehiculos': 'max'
}).reset_index()

operativas_sobre_pasajeros = resumen_operativas[resumen_operativas['porc_ocupacion_pasajeros'] > 100].copy()
operativas_sobre_vehiculos = resumen_operativas[resumen_operativas['porc_ocupacion_vehiculos'] > 100].copy()

operativas_sobre_pasajeros['exceso_pasajeros'] = (operativas_sobre_pasajeros['porc_ocupacion_pasajeros'] - 100).round(1)
operativas_sobre_vehiculos['exceso_vehiculos'] = (operativas_sobre_vehiculos['porc_ocupacion_vehiculos'] - 100).round(1)

summary_pasajeros = operativas_sobre_pasajeros.groupby('buq_nombre').agg(
    operativas_sobre100_pasajeros=('con_bacoope', lambda x: list(x)),
    count_operativas_pasajeros=('con_bacoope', 'nunique'),
    exceso_pasajeros=('exceso_pasajeros', lambda x: [f"{round(v, 1)}%" for v in list(x)])
).reset_index()

summary_vehiculos = operativas_sobre_vehiculos.groupby('buq_nombre').agg(
    operativas_sobre100_vehiculos=('con_bacoope', lambda x: list(x)),
    count_operativas_vehiculos=('con_bacoope', 'nunique'),
    exceso_vehiculos=('exceso_vehiculos', lambda x: [f"{round(v, 1)}%" for v in list(x)])
).reset_index()

summary_total = pd.merge(summary_pasajeros, summary_vehiculos, on='buq_nombre', how='outer')

summary_total['count_operativas_pasajeros'] = summary_total['count_operativas_pasajeros'].fillna(0)
summary_total['count_operativas_vehiculos'] = summary_total['count_operativas_vehiculos'].fillna(0)
summary_total['operativas_sobre100_pasajeros'] = summary_total['operativas_sobre100_pasajeros'].apply(lambda x: x if isinstance(x, list) else [])
summary_total['operativas_sobre100_vehiculos'] = summary_total['operativas_sobre100_vehiculos'].apply(lambda x: x if isinstance(x, list) else [])
summary_total['exceso_pasajeros'] = summary_total['exceso_pasajeros'].apply(lambda x: x if isinstance(x, list) else [])
summary_total['exceso_vehiculos'] = summary_total['exceso_vehiculos'].apply(lambda x: x if isinstance(x, list) else [])

print("Resumen de operativas por buque que superan la capacidad y el exceso de ocupacion (redondeado a 1 decimal y con '%'):")
display(summary_total)

In [ ]:

df_copy["con_fecha"] = pd.to_datetime(df_copy["con_fecha"], errors="coerce")
df_copy["day_of_week"] = df_copy["con_fecha"].dt.dayofweek + 1
df_copy["day_of_week"].value_counts(dropna=False)

In [ ]:
valores_unicos = df_copy["day_of_week"].unique()
print(valores_unicos)

In [ ]:

df_copy.sample(5)

In [ ]:

df_copy["is_weekend"] = df_copy["day_of_week"].isin([6, 7]).astype(int)
df_copy.sample(5)

In [ ]:

df_copy["month"] = df_copy["con_fecha"].dt.month
df_copy.sample(3)

In [ ]:
df_copy["week_of_year"] = df_copy["con_fecha"].dt.isocalendar().week.astype(int)
df_copy.sample(3)

In [ ]:

import numpy as np
conditions = [
    df_copy["month"].isin([12, 1, 2]),  
    df_copy["month"].isin([3, 4, 5]),   
    df_copy["month"].isin([6, 7, 8]),    
    df_copy["month"].isin([9, 10, 11])   
]
choices = [1, 2, 3, 4]

df_copy["season"] = np.select(conditions, choices, default=np.nan).astype(int)
df_copy.sample(5)

In [ ]:
import pandas as pd

festivos_nacionales = pd.to_datetime([
    # 2022
    "2022-01-01",  # Año Nuevo
    "2022-01-06",  # Epifanía del Señor (Reyes)
    "2022-04-15",  # Viernes Santo
    "2022-08-15",  # Asunción de la Virgen
    "2022-10-12",  # Fiesta Nacional de España
    "2022-11-01",  # Todos los Santos
    "2022-12-06",  # Día de la Constitución Española
    "2022-12-08",  # Inmaculada Concepción
    "2022-12-25",  # Natividad del Señor (Navidad)

    # 2023
    "2023-01-06",  # Epifanía del Señor (Reyes)
    "2023-04-07",  # Viernes Santo
    "2023-05-01",  # Día del Trabajo
    "2023-08-15",  # Asunción de la Virgen
    "2023-10-12",  # Fiesta Nacional de España
    "2023-11-01",  # Todos los Santos
    "2023-12-06",  # Día de la Constitución Española
    "2023-12-08",  # Inmaculada Concepción
    "2023-12-25",  # Natividad del Señor (Navidad)

    # 2024
    "2024-01-01",  # Año Nuevo
    "2024-01-06",  # Epifanía del Señor (Reyes)
    "2024-03-29",  # Viernes Santo
    "2024-05-01",  # Día del Trabajo
    "2024-08-15",  # Asunción de la Virgen
    "2024-10-12",  # Fiesta Nacional de España
    "2024-11-01",  # Todos los Santos
    "2024-12-06",  # Día de la Constitución Española
    "2024-12-25",  # Natividad del Señor (Navidad)
])

# Festivos locales en Algeciras (ejemplo aproximado)
festivos_locales = pd.to_datetime([
    "2022-02-28",  # Dia de Andalucia
    "2022-04-14",  # Jueves Santo
    "2022-05-02",  # Lunes siguiente al día del Trabajo
    "2022-06-22",  # Feria Real de Algeciras
    "2022-07-18",  # Festividad de Nuestra Señora del Carmen
    "2022-12-26",  # Lunes siguiente a Navidad

    "2023-01-02",  # Día siguiente a Año Nuevo
    "2023-02-28",  # Día de Andalucía
    "2023-04-06",  # Jueves Santo
    "2023-06-21",  # Miércoles de la Feria Real de Algeciras
    "2023-07-17",  # Lunes posterior a la Festividad de Nuestra Señora del Carmen.​

    "2024-02-28",  # Día de Andalucía
    "2024-03-28",  # Jueves Santo
    "2024-06-26",  # Miércoles de Feria Real de Algeciras
    "2024-07-16",  # Festividad de Nuestra Señora del Carmen
    "2024-12-09",  # Lunes siguiente a la Inmaculada Concepción
])

festivos_nacionales_set = set(festivos_nacionales)
festivos_locales_set = set(festivos_locales)

df_copy["fecha_sin_hora"] = df_copy["con_fecha"].dt.normalize()

df_copy["is_festivo_nacional"] = df_copy["fecha_sin_hora"].isin(festivos_nacionales_set).astype(int)

df_copy["is_festivo_local"] = df_copy["fecha_sin_hora"].isin(festivos_locales_set).astype(int)

In [ ]:

df_copy.groupby(['is_festivo_nacional', 'is_festivo_local']).sample(n=3, random_state=42)[["con_fecha", "is_festivo_nacional", "is_festivo_local"]]

In [ ]:
import pandas as pd
fechas_eid_aladha = pd.to_datetime([
    "2022-07-09",
    "2023-06-28",
    "2024-06-16"
])

set_eid_aladha = set(fechas_eid_aladha)

df_copy["is_eid_aladha"] = df_copy["fecha_sin_hora"].isin(set_eid_aladha).astype(int)

df_copy["is_eid_aladha_prev"] = df_copy["fecha_sin_hora"].apply(
    lambda x: any((x >= fecha - pd.Timedelta(days=7)) and (x < fecha) for fecha in fechas_eid_aladha)
).astype(int)

df_copy["is_eid_aladha_post"] = df_copy["fecha_sin_hora"].apply(
    lambda x: any((x > fecha) and (x <= fecha + pd.Timedelta(days=7)) for fecha in fechas_eid_aladha)
).astype(int)

In [ ]:

df_eid_aladha = df_copy[
    (df_copy["is_eid_aladha"] == 1) |
    (df_copy["is_eid_aladha_prev"] == 1) |
    (df_copy["is_eid_aladha_post"] == 1)
][["fecha_sin_hora", "is_eid_aladha_prev", "is_eid_aladha", "is_eid_aladha_post"]]

df_eid_aladha = df_eid_aladha.drop_duplicates().sort_values("fecha_sin_hora")

print(df_eid_aladha.to_string(index=False))

In [ ]:

import pandas as pd

fechas_eid_alfitr = pd.to_datetime([
    "2022-05-02",
    "2023-04-21",
    "2024-04-10"
])
set_eid_alfitr = set(fechas_eid_alfitr)

df_copy["is_eid_alfitr"] = df_copy["fecha_sin_hora"].isin(set_eid_alfitr).astype(int)


df_copy["is_eid_alfitr_prev"] = df_copy["fecha_sin_hora"].apply(
    lambda x: any((x >= fecha - pd.Timedelta(days=7)) and (x < fecha) for fecha in fechas_eid_alfitr)
).astype(int)


df_copy["is_eid_alfitr_post"] = df_copy["fecha_sin_hora"].apply(
    lambda x: any((x > fecha) and (x <= fecha + pd.Timedelta(days=7)) for fecha in fechas_eid_alfitr)
).astype(int)

In [ ]:

df_eid_alfitr = df_copy[
    (df_copy["is_eid_alfitr"] == 1) |
    (df_copy["is_eid_alfitr_prev"] == 1) |
    (df_copy["is_eid_alfitr_post"] == 1)
][["fecha_sin_hora", "is_eid_alfitr_prev", "is_eid_alfitr", "is_eid_alfitr_post"]]

df_eid_alfitr = df_eid_alfitr.drop_duplicates().sort_values("fecha_sin_hora")

print(df_eid_alfitr.to_string(index=False))


In [ ]:
import pandas as pd
fechas_mawlid_nabi = pd.to_datetime([
    "2022-10-08",
    "2023-09-27",
    "2024-09-15"
])

set_mawlid_nabi = set(fechas_mawlid_nabi)
df_copy["is_mawlid_nabi"] = df_copy["fecha_sin_hora"].isin(set_mawlid_nabi).astype(int)
df_mawlid_nabi = df_copy[df_copy["is_mawlid_nabi"] == 1][["fecha_sin_hora", "is_mawlid_nabi"]].drop_duplicates()
print(df_mawlid_nabi.to_string(index=False))

In [ ]:

df_copy.drop(columns=["fecha_sin_hora"], inplace=True)

In [ ]:
display(df_copy.head())
df_copy.info()

In [ ]:
df_copy['con_fecha'] = pd.to_datetime(df_copy['con_fecha'], errors='coerce')
df_copy['hora_embarque'] = df_copy['con_fecha'].dt.hour.astype('int')

In [53]:
df_copy['agrupacion_billete'] = np.where(df_copy['con_tipobillete'].isin([0, 5, 6, 7, 8, 9]), 0,
                              np.where(df_copy['con_tipobillete'].isin([1, 2, 3]), 1, 2))

In [ ]:
display(df_copy.head())
df_copy.info()

In [ ]:

df_copy.to_csv("dataset_empresa_03_pos_EDA_0.csv", index=False)

from google.colab import files
files.download("dataset_empresa_03_pos_EDA_0.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>